<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/AMPDs/training/nilm_nn_cwe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!mkdir my_net

In [0]:
mv checkpoint cwe_stacked_model.ckpt.data-00000-of-00001 cwe_stacked_model.ckpt.index cwe_stacked_model.ckpt.meta my_net

In [0]:
rm -r cwe_rnn

In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np
import io
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from google.colab import files
import matplotlib.pyplot as plt

In [0]:
def sliding_window_transformation(dataset, window_size=10):
  '''
  A function to transform time-series data into tabular by sliding window
  Args:
    dataset: the target time-series data.
    window_size: default size=10
  Returns: the tansformed tabular data
  '''
  dataX = []
  for i in range(len(dataset)-window_size+1):
      a = dataset[i:(i+window_size)]
      dataX.append(a)
  return np.array(dataX)

In [0]:
non_time_data = pd.read_csv('Electricity_WHE.csv')['unix_ts']

In [0]:
CWE_data = pd.read_csv('Electricity_CWE.csv')['P']

In [10]:
nn_data = pd.read_csv('nn_data.csv')
print(nn_data.shape)

(1048141, 3)


In [0]:
scaler = MinMaxScaler()
scaler.fit(nn_data.astype(float))
nn_data = scaler.transform(nn_data)

import pickle
file_name='scaler_stacked_nn_cwe'
fileObject = open(file_name,'wb')
pickle.dump(scaler,fileObject)
fileObject.close()

In [0]:
window_size=60
non_time_data = pd.to_datetime(non_time_data,unit='s')
whe_month = non_time_data.dt.month.as_matrix()[window_size-1:].reshape([-1,1])
whe_hour = non_time_data.dt.hour.as_matrix()[window_size-1:].reshape([-1,1])
whe_weekday = non_time_data.dt.dayofweek.as_matrix()[window_size-1:].reshape([-1,1])
non_time = np.hstack([whe_month,whe_hour,whe_weekday])
non_time = np.concatenate([non_time[0:472500,:],non_time[475500:,:]],axis=0)
data_arr = np.hstack([nn_data,non_time])


In [0]:

dataX = pd.get_dummies(pd.DataFrame(data_arr),columns=[3,4,5]).values
cwe_Y = CWE_data.as_matrix()[window_size-1:].reshape([-1,1])
cwe_Y = np.concatenate([cwe_Y[0:472500,:],cwe_Y[475500:,:]],axis=0)
categorized_cwe_Y = np.ones(cwe_Y.shape)*2
categorized_cwe_Y[[np.where(cwe_Y==0)[0]],:]=0
categorized_cwe_Y[[np.where((cwe_Y>0)&(cwe_Y<=300))[0]],:]=1
encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
cweY_1hot = encoder.fit_transform(categorized_cwe_Y)

In [14]:
max_indice = np.argmax(cweY_1hot,1)
df = pd.Series(max_indice)
df.value_counts()

0    1018321
1      26327
2       3493
dtype: int64

In [0]:
X_train, X_test, y_train, y_test = train_test_split(dataX, cweY_1hot, test_size=0.01, shuffle=True)

In [0]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import OneHotEncoder
tl = RandomUnderSampler(sampling_strategy={0:3000,1:3000,2:3000})
X, y = tl.fit_resample(X_train, y_train)
whole_train = np.concatenate([X,y],axis=1)

In [0]:
#define the Feedforward Model
'''Hyper parameters for deep learning'''
LR = 0.001               # learning rate
batch_size=128
'''Feedforward Model Definition'''
tf.reset_default_graph()

''''''
#define inputs
tf_x = tf.placeholder(tf.float32, [None, 46],name='ff_x')
tf_y = tf.placeholder(tf.int32, [None, 3],name='ff_y')

l1 = tf.layers.dense(tf_x,64,activation=tf.nn.leaky_relu,name='ff_l1')
l2 = tf.layers.dense(l1,32,activation=tf.nn.leaky_relu,name='ff_l2')
l3 = tf.layers.dense(l2,16,activation=tf.nn.leaky_relu,name='ff_l3')
l4 = tf.layers.dense(l3,8,activation=tf.nn.leaky_relu,name='ff_l4')
pred = tf.layers.dense(l4,3,activation=tf.nn.relu,name='ff_pred')

with tf.name_scope('loss'):
    cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_y, logits=pred) 
    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("loss",tensor=loss)

train_op = tf.train.AdamOptimizer(LR).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf_y, axis=1), tf.argmax(pred, axis=1)), tf.float32))

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) 
saver = tf.train.Saver()


In [35]:
sess = tf.Session()
#sess.run(init_op)
saver.restore(sess, 'my_net/cwe_stacked_model.ckpt')

flag = False
for j in range(0,2000):
    print('###iteration: '+str(j)+'###')
    batch_index = np.random.choice(9000,batch_size)
    batch_train = whole_train[batch_index,:]
    batch_X = batch_train[:,:-3].reshape([-1,46])
    batch_y = batch_train[:,-3:]
    sess.run(train_op,{tf_x:batch_X , tf_y:batch_y})
    cost_ = sess.run(loss,{tf_x:batch_X, tf_y:batch_y})
    print('train loss= %.4f' % cost_)
    if(j%99==0):
      save_path = saver.save(sess, "my_net/cwe_stacked_model.ckpt")
      
      acc_train = sess.run(accuracy,{tf_x:batch_X, tf_y:batch_y})
      acc_test = sess.run(accuracy,feed_dict={tf_x: X_test.reshape([X_test.shape[0],46]), tf_y:y_test})
      print('train loss= %.4f' % cost_+', Acc=%.2f'% acc_train)
      print('Test Acc=%.2f'% acc_test)
      pre = sess.run(pred,feed_dict={tf_x: batch_X, tf_y:batch_y})
      y_lables_argmax = tf.argmax(tf_y,axis=1)  
      y_pred_argmax = tf.argmax(pre,axis=1)
      confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=3)
      #print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
      print(confusion.eval(session=sess,feed_dict={tf_x: batch_X, tf_y:batch_y}))
pre = sess.run(pred,feed_dict={tf_x: X_test.reshape([X_test.shape[0],46]), tf_y: y_test})
y_lables_argmax = np.argmax(y_test,1)
y_pred_argmax = np.argmax(pre,1)
confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=3)
#print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
print(confusion.eval(session=sess))
sess.close()


INFO:tensorflow:Restoring parameters from my_net/cwe_stacked_model.ckpt
###iteration: 0###
train loss= 0.0708
train loss= 0.0708, Acc=0.98
Test Acc=0.90
[[47  2  0]
 [ 0 42  0]
 [ 0  0 37]]
###iteration: 1###
train loss= 0.1031
###iteration: 2###
train loss= 0.1360
###iteration: 3###
train loss= 0.1045
###iteration: 4###
train loss= 0.1403
###iteration: 5###
train loss= 0.0750
###iteration: 6###
train loss= 0.0861
###iteration: 7###
train loss= 0.1329
###iteration: 8###
train loss= 0.0980
###iteration: 9###
train loss= 0.0758
###iteration: 10###
train loss= 0.0859
###iteration: 11###
train loss= 0.0578
###iteration: 12###
train loss= 0.1007
###iteration: 13###
train loss= 0.0530
###iteration: 14###
train loss= 0.1413
###iteration: 15###
train loss= 0.0759
###iteration: 16###
train loss= 0.1459
###iteration: 17###
train loss= 0.0709
###iteration: 18###
train loss= 0.1497
###iteration: 19###
train loss= 0.1029
###iteration: 20###
train loss= 0.0975
###iteration: 21###
train loss= 0.1132
